# Setting Up the Environment
This section imports necessary libraries and sets up the environment for the notebook.

In [1]:
import nest_asyncio
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

nest_asyncio.apply()

# Connecting to Qdrant
This section establishes a connection to the Qdrant vector database.

In [2]:
import qdrant_client

collection_name="chat_with_docs"

client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

# Instrumentation Setup
This section sets up instrumentation for tracing and monitoring.

In [3]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Loading Documents
This section loads documents from the specified directory.

In [4]:
from llama_index.core import SimpleDirectoryReader

input_dir_path = './docs'

loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

# Checking Loaded Documents
This section checks the type and number of loaded documents.

In [5]:
type(docs), len(docs)


(list, 63)

# Creating a Vector Store Index
This section defines a function to create a vector store index using Qdrant.

In [6]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext

def create_index(documents):

    vector_store = QdrantVectorStore(client=client,
                                     collection_name=collection_name)
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    index = VectorStoreIndex.from_documents(documents,
                                            storage_context=storage_context)
    
    return index

# Setting Up Embedding Model
This section sets up the embedding model for the index.

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                   trust_remote_code=True)

Settings.embed_model = embed_model

index = create_index(docs)

I0000 00:00:1746540167.877371   46210 chttp2_transport.cc:1201] ipv6:%5B::1%5D:4317: Got goaway [11] err=UNAVAILABLE:GOAWAY received; Error code: 11; Debug Text: ping_timeout {grpc_status:14, http2_error:11, created_time:"2025-05-06T19:32:47.875857944+05:30"}
Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 1s.


# Configuring LLM
This section configures the language model for the query engine.

In [8]:
from llama_index.llms.groq import Groq
from llama_index.core.settings import Settings

llm = Groq(model="gemma2-9b-it", request_timeout=120.0)
Settings.llm = llm

# Defining a Prompt Template
This section defines a custom prompt template for the query engine.

In [9]:
from llama_index.core import PromptTemplate

template = """Context information is below:
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner,
              incase you don't know the answer say 'I don't know!'
            
              Query: {query_str}
        
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

# Setting Up Reranking
This section sets up a reranking mechanism for query results.

In [10]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
    top_n=3
)

# Querying the Engine
This section demonstrates how to query the engine with a custom prompt and reranking.

In [11]:
query_engine = index.as_query_engine(similarity_top_k=10,
                                     node_postprocessors=[rerank])

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

response = query_engine.query("What exactly is ReSpAct?")

# Displaying the Response
This section displays the response from the query engine in a markdown format.

In [12]:
from IPython.display import Markdown, display

display(Markdown(str(response)))

ReSpAct is a task-oriented dialogue agent that leverages user interaction to enhance its decision-making capabilities. 

Here's a breakdown:

* **ReSpAct stands for "Reasoning and SpAct"**:  It combines reasoning abilities with interactive strategies.
* **Key Feature: User Interaction**: Unlike purely reasoning-based agents, ReSpAct actively seeks user input and guidance during task execution. This allows it to clarify ambiguities, handle unexpected situations, and learn from user feedback.
* **Benefits**: This interactive approach leads to:
    * **Improved Performance**: ReSpAct often outperforms purely reasoning-based agents (like ReAct) in complex tasks.
    * **Robustness**: It can adapt to unfamiliar scenarios and handle user input that might be incomplete or even contradictory.
    * **Efficiency**: By leveraging user knowledge, ReSpAct can sometimes complete tasks faster and with fewer trial-and-error attempts.

* **How it Works**: ReSpAct uses a language model (like GPT-4o or LLaMA) as its core. It's trained on datasets that include examples of human-agent interactions in various task-oriented domains. During task execution, ReSpAct generates a sequence of actions, thoughts, and dialogue acts. When it encounters uncertainty or needs clarification, it interacts with the user, seeking information or guidance.


Let me know if you have any more questions about ReSpAct!
